# Sales Forecasting

Walmart has been having trouble figuring out how much to buy to restock its products and optimize sales. On the last important dates for the retail sector, the team had problems with purchasing supplies. In some departments, the purchase was above the expected sales and in others there was a lack of products.

The supply team asked us to help them solve their problem in forecasting demand for each store and each department. The team opted to build a regression model to predict sales and thus help the supply team scale purchases.

# 1.0 Data Catalog

**Dataset - Store**<br>
**Store** - Store IDs <br>
**Type** - Store types <br>
**Size** - Store sizes <br> <br>

**Dataset - Features**<br>
**Store** - Store IDs <br>
**Date** - The week <br>
**Temperature** - Average temperature in the region <br>
**Fuel Price** - Cost of fuel in region <br>
**MarkDown 1-5** - Anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA <br>
**CPI** - The customer price index <br>
**Unemployment** - The unemployment rate <br>
**IsHoliday** - Whether the week is a especial holiday week <br>
 <br>

**Dataset - Train/Test**<br>
**Store** - Store IDs <br>
**Dept** - Departament <br>
**Weekly_sales** - Weekly Sales <br>
**Date** - The first date of the week <br>
**IsHoliday** - True or False <br><br>


For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

**Super Bowl:**  <br>
    - 12-Feb-10 <br>
    - 11-Feb-11 <br>
    - 10-Feb-12 <br>
    - 8-Feb-13 <br>
**Labor Day:**  <br>
    - 10-Sep-10 <br>
    - 9-Sep-11 <br>
    - 7-Sep-12 <br>
    - 6-Sep-13 <br>
**Thanksgiving:**  <br>
    - 26-Nov-10 <br>
    - 25-Nov-11 <br>
    - 23-Nov-12 <br>
    - 29-Nov-13 <br>
**Christmas:**  <br>
    - 31-Dec-10 <br>
    - 30-Dec-11 <br>
    - 28-Dec-12 <br>
    - 27-Dec-13 <br>

# 2.0 Libraries

In [9]:
import pandas as pd
import numpy as np

# 3.0 Import Data

In [10]:
df_stores = pd.read_csv('stores.csv')
df_stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [13]:
df_features = pd.read_csv('features.csv')
df_features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


In [11]:
df_train = pd.read_csv('train.csv')
df_train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [12]:
df_test = pd.read_csv('test.csv')
df_test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


# WIP